In [1]:
#https://www.matecdev.com/posts/landsat-sentinel-aws-s3-python.html

from pystac_client import Client

LandsatSTAC = Client.open("https://landsatlook.usgs.gov/stac-server", headers=[])

for collection in LandsatSTAC.get_collections():
    print(collection)

<CollectionClient id=landsat-c2l2-sr>
<CollectionClient id=landsat-c2l2-st>
<CollectionClient id=landsat-c2ard-st>
<CollectionClient id=landsat-c2l2alb-bt>
<CollectionClient id=landsat-c2l3-fsca>
<CollectionClient id=landsat-c2ard-bt>
<CollectionClient id=landsat-c2l1>
<CollectionClient id=landsat-c2l3-ba>
<CollectionClient id=landsat-c2l2alb-st>
<CollectionClient id=landsat-c2ard-sr>
<CollectionClient id=landsat-c2l2alb-sr>
<CollectionClient id=landsat-c2l2alb-ta>
<CollectionClient id=landsat-c2l3-dswe>
<CollectionClient id=landsat-c2ard-ta>


In [6]:
import satsearch

SentinelSTAC = satsearch.Search.search( url = "https://earth-search.aws.element84.com/v0" )
print("Found " + str(SentinelSTAC.found()) + "items")

Found 56901171items


In [4]:
#from json import load

#file_path = "my-area.geojson"
#file_content = load(open(file_path))
#geometry = file_content["features"][0]["geometry"]

In [7]:
timeRange = '2019-06-01/2021-06-01'

In [9]:
# Define a small area
def BuildSquare(lon, lat, delta):
    c1 = [lon + delta, lat + delta]
    c2 = [lon + delta, lat - delta]
    c3 = [lon - delta, lat - delta]
    c4 = [lon - delta, lat + delta]
    geometry = {"type": "Polygon", "coordinates": [[ c1, c2, c3, c4, c1 ]]}
    return geometry

geometry = BuildSquare(-59.346271, -34.233076, 0.04)

In [12]:
# Now we can query the STACT server
LandsatSearch = LandsatSTAC.search ( 
    intersects = geometry,
    datetime = timeRange,
    query =  ['eo:cloud_cover95'],
    collections = ["landsat-c2l2-sr"] )

Landsat_items = [i.to_dict() for i in LandsatSearch.get_items()]
print(f"{len(Landsat_items)} Landsat scenes fetched")

193 Landsat scenes fetched


In [15]:
Landsat_items[0]['assets']

{'thumbnail': {'href': 'https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2021/225/084/LC08_L2SP_225084_20210528_20210607_02_T1/LC08_L2SP_225084_20210528_20210607_02_T1_thumb_small.jpeg',
  'type': 'image/jpeg',
  'title': 'Thumbnail image',
  'alternate': {'s3': {'storage:platform': 'AWS',
    'storage:requester_pays': True,
    'href': 's3://usgs-landsat/collection02/level-2/standard/oli-tirs/2021/225/084/LC08_L2SP_225084_20210528_20210607_02_T1/LC08_L2SP_225084_20210528_20210607_02_T1_thumb_small.jpeg'}},
  'file:checksum': '13406cddb575756e94b987912d7e4849eecaaf9b8196fcb113e994d78ecd311fb8110340e8ee226aeb2abecad3992565ed2c6e7b1a1d90df2435d3540d5276d2251e',
  'roles': ['thumbnail']},
 'reduced_resolution_browse': {'href': 'https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2021/225/084/LC08_L2SP_225084_20210528_20210607_02_T1/LC08_L2SP_225084_20210528_20210607_02_T1_thumb_large.jpeg',
  'type': 'image/jpeg',
  'title': 'Reduced resolution

In [16]:
#print the URLs of the red band of the Landsat
for item in Landsat_items:
    red_href = item['assets']['red']['href']
    red_s3 = item['assets']['red']['alternate']['s3']['href']
    print(red_href)    
    print(red_s3)

https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2021/225/084/LC08_L2SP_225084_20210528_20210607_02_T1/LC08_L2SP_225084_20210528_20210607_02_T1_SR_B4.TIF
s3://usgs-landsat/collection02/level-2/standard/oli-tirs/2021/225/084/LC08_L2SP_225084_20210528_20210607_02_T1/LC08_L2SP_225084_20210528_20210607_02_T1_SR_B4.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/etm/2021/226/084/LE07_L2SP_226084_20210527_20210622_02_T1/LE07_L2SP_226084_20210527_20210622_02_T1_SR_B3.TIF
s3://usgs-landsat/collection02/level-2/standard/etm/2021/226/084/LE07_L2SP_226084_20210527_20210622_02_T1/LE07_L2SP_226084_20210527_20210622_02_T1_SR_B3.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/etm/2021/225/084/LE07_L2SP_225084_20210520_20210615_02_T1/LE07_L2SP_225084_20210520_20210615_02_T1_SR_B3.TIF
s3://usgs-landsat/collection02/level-2/standard/etm/2021/225/084/LE07_L2SP_225084_20210520_20210615_02_T1/LE07_L2SP_225084_20210520_20210615_02_T1_SR_B3.TIF


In [20]:
SentinelSearch = satsearch.Search.search( 
    url = "https://earth-search.aws.element84.com/v0",
    intersects = geometry,
    datetime = timeRange,
    collections = ['sentinel-s2-l2a'] )

Sentinel_items = SentinelSearch.items()
print(Sentinel_items.summary())

for item in Sentinel_items:
    red_s3 = item.assets['B04']['href']
    #print(red_s3)

Items (143):
date                      id                        
2021-05-28                S2B_21HTC_20210528_0_L2A  
2021-05-23                S2A_21HTC_20210523_0_L2A  
2021-05-18                S2B_21HTC_20210518_0_L2A  
2021-05-13                S2A_21HTC_20210513_0_L2A  
2021-05-08                S2B_21HTC_20210508_0_L2A  
2021-05-03                S2A_21HTC_20210503_0_L2A  
2021-04-28                S2B_21HTC_20210428_0_L2A  
2021-04-23                S2A_21HTC_20210423_0_L2A  
2021-04-18                S2B_21HTC_20210418_0_L2A  
2021-04-13                S2A_21HTC_20210413_0_L2A  
2021-04-08                S2B_21HTC_20210408_0_L2A  
2021-04-03                S2A_21HTC_20210403_0_L2A  
2021-03-29                S2B_21HTC_20210329_0_L2A  
2021-03-24                S2A_21HTC_20210324_0_L2A  
2021-03-19                S2B_21HTC_20210319_0_L2A  
2021-03-14                S2A_21HTC_20210314_0_L2A  
2021-03-09                S2B_21HTC_20210309_0_L2A  
2021-03-04                S2A_21H

In [ ]:
https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a

In [21]:
print(red_s3)

s3://sentinel-s2-l2a/tiles/21/H/TC/2019/6/3/0/R10m/B04.jp2


In [8]:
print(item)
item.assets.keys()


S2A_21HTC_20190603_0_L2A


dict_keys(['thumbnail', 'overview', 'info', 'metadata', 'visual', 'visual_20m', 'visual_60m', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL'])

In [30]:
import os
import boto3
import rasterio as rio
import xarray as xr
from rasterio.session import AWSSession
from rasterio.plot import show

os.environ['CURL_CA_BUNDLE'] = '/etc/ssl/certs/ca-certificates.crt'

print("Creating AWS Session")
aws_session = rio.session.AWSSession(boto3.Session(), requester_pays=True)

Creating AWS Session


In [31]:
from pyproj import Transformer

def getSubset(geotiff_file, bbox):
    with rio.Env(aws_session):
        with rio.open(geotiff_file) as geo_fp:
            # Calculate pixels with PyProj 
            Transf = Transformer.from_crs("epsg:4326", geo_fp.crs) 
            lat_north, lon_west = Transf.transform(bbox[3], bbox[0])
            lat_south, lon_east = Transf.transform(bbox[1], bbox[2]) 
            x_top, y_top = geo_fp.index( lat_north, lon_west )
            x_bottom, y_bottom = geo_fp.index( lat_south, lon_east )
            # Define window in RasterIO
            window = rio.windows.Window.from_slices( ( x_top, x_bottom ), ( y_top, y_bottom ) )                
            # Actual HTTP range request
            subset = geo_fp.read(1, window=window)
    return subset

In [17]:
# Now process the data (locally)
def plotNDVI(nir,red,filename, save=False):
    ndvi = (nir-red)/(nir+red)
    ndvi[ndvi>1] = 1
    plt.imshow(ndvi)
    if save:
        plt.savefig(filename)
        plt.close()

In [23]:
from rasterio.features import bounds
import matplotlib.pyplot as plt

bbox = bounds(geometry)

In [32]:
for i,item in enumerate(Sentinel_items):
    red_s3 = item.assets['B04']['href']
    nir_s3 = item.assets['B08']['href']
    date = item.properties['datetime'][0:10]
   # print("Sentinel item number " + str(i) + "/" + str(len(Sentinel_items)) + " " + date)
   # red = getSubset(red_s3, bbox)
   # nir = getSubset(nir_s3, bbox)
    #plotNDVI(nir,red,"sentinel/" + date + "_ndvi.png", save=False)

In [36]:
rio.open(red_s3)

RasterioIOError: '/vsis3/sentinel-s2-l2a/tiles/21/H/TC/2019/6/3/0/R10m/B04.jp2' does not exist in the file system, and is not recognized as a supported dataset name.

In [33]:
with rio.Env(aws_session):
        with rio.open(red_s3) as geo_fp:
            Transf = Transformer.from_crs("epsg:4326", geo_fp.crs)

RasterioIOError: '/vsis3/sentinel-s2-l2a/tiles/21/H/TC/2019/6/3/0/R10m/B04.jp2' does not exist in the file system, and is not recognized as a supported dataset name.

In [ ]:



for i,item in enumerate(Landsat_items):
    red_s3 = item['assets']['red']['alternate']['s3']['href']
    nir_s3 = item['assets']['nir08']['alternate']['s3']['href']
    date = item['properties']['datetime'][0:10]
    print("Landsat item number " + str(i) + "/" + str(len(Landsat_items)) + " " + date)
    red = getSubset(red_s3, bbox)
    nir = getSubset(nir_s3, bbox)
    plotNDVI(nir,red,"landsat/" + date + "_ndvi.png")

